This code implements the market clearing model for a 3-bus power grid with spatial virtual links. Each bus is associated with one data center that receives and processes loads. All three data centers are owned by the same entity. The DC power flow constraints are accomodated.

In [1]:
using JuMP, Gurobi

In [16]:
# scenario_num takes integer value from 1 to 7. Each scenario corresponds to different virtual shift capacity
scenario_num = 7;

In [17]:
# Node parameters
N = 3                    # Number of nodes
α_p = [10,20,10]         # bidding prices of supply
α_d = [40,30,40]         # bidding prices of demand
P = [50,30,50]           # supply capacity
D = [40,45,40]           # load requested
d_max = D .+ 20          # data center capacity 

# Connection parameters
A = [[0 1 1];
     [1 0 1];
     [1 1 0]]             # adjacency matrix of transmission lines

V = [[0 1 0];
     [1 0 0];
     [0 0 0]]             # adjacency matrix of virtual links

F = [[0 5 10];
     [5 0 10];
     [10 10 0]]           # capacity matrix of transmission lines

α_f = 2 * (copy(F).>0)    # bidding cost of transmission


#θ_diff_lim = 15;


Del = zeros(N,N)          # capacity matrix of virtual links
                          # scenario 1: no spatial shifts
if scenario_num == 2      # scenario 2: δ_{12} = 5
    Del[1,2] = 5
    Del[2,1] = 5
elseif scenario_num == 3  # scenario 3: δ_{12} = 15
    Del[1,2] = 15
    Del[2,1] = 15
elseif scenario_num == 4  # scenario 4: δ_{12} = δ_{23} = 15
    Del[1,2] = 15
    Del[2,1] = 15
    Del[1,3] = 15
    Del[3,1] = 15
    Del[2,3] = 15
    Del[3,2] = 15
elseif scenario_num == 5  # scenario 5: δ_{12} = δ_{23} = δ_{13} = 100
    Del[1,2] = 100
    Del[2,1] = 100
    Del[2,3] = 100
    Del[3,2] = 100
    Del[1,3] = 100
    Del[3,1] = 100
elseif scenario_num == 6  # scenario 6: δ_{12} = δ_{23} = 15, but bidding cost is 1 instead of 3 (below)
    Del[1,2] = 15
    Del[2,1] = 15
    Del[1,3] = 15
    Del[3,1] = 15
    Del[2,3] = 15
    Del[3,2] = 15
elseif scenario_num == 7  # scenario 6: δ_{12} = δ_{23} = 15, but bidding cost is 0 instead of 3 (below)
    Del[1,2] = 15
    Del[2,1] = 15
    Del[1,3] = 15
    Del[3,1] = 15
    Del[2,3] = 15
    Del[3,2] = 15
end

α_del =zeros(N,N)         # virtual shift bidding costs
if scenario_num == 6
    α_del = 1 * (copy(Del).>0)
elseif scenario_num < 6
    α_del = 3 * (copy(Del).>0)
end

B = 0.5;                 # susceptance for all physical transmissions

In [18]:
# Optimizaiton formulation
# The implemented model is a reformulation equivalent to the model presented in the paper
# in order to convert the original problem with absolute values into a linear program

m = Model(with_optimizer(Gurobi.Optimizer, OutputFlag=0))
@variable(m, p[1:N] >= 0)                         # supply
@variable(m, d[1:N] >= 0)                         # demand
@variable(m, f[1:N,1:N] >= 0)                     # flow
@variable(m, δ[1:N,1:N] >= 0)                     # virtual shift
@variable(m, θ[1:N])                              # phase angles

# Flow and virtual link constraints
for i in 1:N, j in 1:N
    @constraint(m, f[i,j] <= F[i,j])              # flow capacity constraints
    @constraint(m, δ[i,j] <= Del[i,j])            # virtual shift capacity constraints

    # DC power flow constraints
    if (F[i,j] > 0)
        @constraint(m, f[i,j] - f[j,i] == B*(θ[j] - θ[i]))
    end
end

# Supply and demand capacity constraints
for i in 1:N
    @constraint(m, p[i] <= P[i])
    @constraint(m, d[i] <= D[i])
    @constraint(m, d[i] - sum(δ[i,:]) + sum(δ[:,i]) <= d_max[i])
    @constraint(m, d[i] - sum(δ[i,:]) + sum(δ[:,i]) >= 0)
end

# Node balance constraint
bals = Dict()
for i in 1:N
    bals[i] = @constraint(m, p[i] - d[i] - sum(f[i,:]) + sum(f[:,i]) + sum(δ[i,:]) - sum(δ[:,i]) == 0)
end

# Limit on phase angles differences
#=
for i in 1:N, j in 1:N
    if (F[i,j] > 0)
        @constraint(m, θ[i] - θ[j] <= θ_diff_lim)
        @constraint(m, θ[j] - θ[i] <= θ_diff_lim)        
    end
end
=#

# Objecitve: social welfare
@objective(m, Min, sum(α_p.*p - α_d.*d) + sum(α_f.*f) + sum(α_del.*δ));


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only


In [19]:
optimize!(m)
prices = zeros(N)
for i in 1:N
    prices[i] = dual(bals[i])
end

# reformulate flows and virtual shifts
flows = zeros(1,3)
vshift = zeros(1,3)
start_pt = [1,1,2]
end_pt = [2,3,3]
for i in 1:3
    flows[i] = value.(f)[start_pt[i],end_pt[i]] - value.(f)[end_pt[i],start_pt[i]]
    vshift[i] = value.(δ)[start_pt[i],end_pt[i]] - value.(δ)[end_pt[i],start_pt[i]]
end

# Print info
println("Total social welfare: ", -objective_value(m))
println("Amount of cleared loads: ", value.(d))
println("Flows: ", flows)
println("Amount of cleared supplies: ", value.(p))
println("Virtual flows: ", vshift)
println("LMP: ", prices)


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only
Total social welfare: 3050.0
Amount of cleared loads: [40.0, 45.0, 40.0]
Flows: [0.0 0.0 0.0]
Amount of cleared supplies: [50.0, 25.0, 50.0]
Virtual flows: [-5.0 -5.0 15.0]
LMP: [20.0, 20.0, 20.0]


In [20]:
value.(θ)

3-element Array{Float64,1}:
 0.0
 0.0
 0.0